# TravelTide


## Overview
This report offers a deep dive into your customers' travel behaviors and preferences. By analyzing your extensive data, we aim to uncover what drives your travelers and how to keep them engaged. Our approach segments customers into distinct groups or "traveler tribes," allowing for the customization of rewards and incentives that feel uniquely tailored to each group. This analysis distills vast data into clear, actionable metrics that shed light on travel patterns, service preferences, and responsiveness to promotions. The ultimate goal is to enhance customer loyalty and drive business success by keeping your travelers engaged with your services.

## Approach and Data Analysis
This section outlines how customer segmentation can be applied within TravelTide to create targeted rewards and incentives. By categorizing customers based on their behaviors and preferences, we can significantly improve customer satisfaction and loyalty.

Our objective is to leverage segmentation to deliver personalized perks that boost loyalty, engagement, and overall satisfaction. We aim to offer at least five specific perks:

## Reward Offerings:

-  Exclusive Discounts
- 1 Night Free Hotel with Flight
- No Cancellation Fees
- Free Hotel Meal
- Free Checked Bag

## Data Processing and Metrics Creation
Starting with a dataset of 50,570 sessions, we distilled the information into 5,998 unique customers, using PostgreSQL to generate key metrics. This process was essential for simplifying the raw data into manageable and insightful variables.

Note: The SQL script file is in SQL folder.

## Key Metrics for Analysis
To determine the most suitable perks for each customer, we focused on the following aggregated metrics:

- Flight/Hotel/Both Preferences
- Age
- Age Group
- Conversion Rate
- Cancellation Rate
- Total Sessions
- Total Trips Booked
- Engagement (Click Efficiency)
- User Activity Level
- Average Checked Bags
- Discount Responsiveness
- Proportions of Discounts (Flights/Hotels/Both)
- Average Offers Received
- Flight Hunter Index
- Hotel Hunter Index

Geo Distance is calculated in SQL.

## Packages need to be installed
- pip install scikit-learn



In [1]:
import pandas as pd
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px

# custom functions 
import Support.dbSupport as dbs
import Support.calSupport as cs

## Db Connection setup

In [2]:
current_dir = os.getcwd()

In [3]:
dbs.check_tables()

['hotels', 'users', 'flights', 'sessions']

In [4]:
dbs.table_row_count()

hotels: 1918617 records
users: 1020926 records
flights: 1901038 records
sessions: 5408063 records


In [5]:

#All_info_combined.sql
sql_file_path = os.path.join(current_dir, 'SQL', 'All_info_combined.sql')

In [6]:
CombainedData = dbs.execute_sql_file(sql_file_path)


In [7]:
CombainedData.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,age,age_group,latest_session,...,both_discount_proportion,discount_responsiveness,total_hotel_usd_spent,total_flight_usd_spent,total_usd_spent,total_nights,scaled_hotel_ads,ads_per_km,scaled_ads_per_km,hotel_hunter_index
0,23557,1958-12-08,F,True,False,usa,new york,65.0,65+,2023-07-14,...,0.0,0.083333,563.0,1344.96,1907.96,22.0,0.164438,0.000007,0.002772,0.004796
1,94883,1972-03-16,F,True,False,usa,kansas city,52.0,45-54,2023-05-28,...,0.0,0.027778,230.0,5354.86,5584.86,1.0,0.000000,0.000002,0.000703,0.000000
2,101486,1972-12-07,F,True,True,usa,tacoma,51.0,45-54,2023-07-18,...,0.0,0.000000,1195.0,5994.28,7189.28,38.0,0.000000,0.000011,0.004808,0.000000
3,101961,1980-09-14,F,True,False,usa,boston,43.0,35-44,2023-06-22,...,0.0,0.011905,1052.0,1929.20,2981.20,22.0,0.000000,0.000019,0.007858,0.000000
4,106907,1978-11-17,F,True,True,usa,miami,45.0,45-54,2023-07-27,...,0.0,0.017857,1185.0,27969.63,29154.63,34.0,0.113616,0.000001,0.000589,0.001623


In [8]:
CombainedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     5998 non-null   int64  
 1   birthdate                   5998 non-null   object 
 2   gender                      5998 non-null   object 
 3   married                     5998 non-null   bool   
 4   has_children                5998 non-null   bool   
 5   home_country                5998 non-null   object 
 6   home_city                   5998 non-null   object 
 7   age                         5998 non-null   float64
 8   age_group                   5998 non-null   object 
 9   latest_session              5998 non-null   object 
 10  total_trips                 5998 non-null   int64  
 11  total_cancellations         5998 non-null   int64  
 12  total_sessions              5998 non-null   int64  
 13  total_cancellation_rate     5998 

In [9]:
CombainedData[['total_trips','total_cancellations','total_cancellation_rate','flight_discount_proportion','hotel_discount_proportion','both_discount_proportion','discount_responsiveness']]

,total_trips,total_cancellations,total_cancellation_rate,flight_discount_proportion,hotel_discount_proportion,both_discount_proportion,discount_responsiveness
0,5,1,0.20,0.083333,0.166667,0.0,0.083333
1,3,0,0.00,0.083333,0.166667,0.0,0.027778
2,5,1,0.20,0.307692,0.000000,0.0,0.000000
3,7,0,0.00,0.250000,0.083333,0.0,0.011905
4,4,1,0.25,0.071429,0.071429,0.0,0.017857
...,...,...,...,...,...,...,...
5993,4,0,0.00,0.125000,0.000000,0.0,0.093750
5994,4,1,0.25,0.125000,0.000000,0.0,0.031250
5995,3,0,0.00,0.375000,0.000000,0.0,0.000000
5996,1,0,0.00,0.250000,0.125000,0.0,0.000000


In [10]:
CombainedData['flight_hunter_index'] = CombainedData['scaled_ads_per_km'] * CombainedData['flight_discount_proportion'] * CombainedData['average_flight_discount']


## User Demographics and Characteristics

### 1. Age Distribution
KPIs:

- Average Age: The mean age of all users.
- Median Age: The middle value when the ages are sorted.
- Age Group Distribution: The percentage of users in different age groups (e.g., 18-25, 26-35, etc.).


In [11]:
# Calculate average and median age
average_age = CombainedData['age'].mean()
median_age = CombainedData['age'].median()

# Calculate age group distribution
age_group_distribution = CombainedData['age_group'].value_counts(normalize=True) * 100

# Display the results
print(f"Average Age: {average_age}")
print(f"Median Age: {median_age}")
print(f"Age Group Distribution:\n{age_group_distribution}")

Average Age: 41.942814271423806
Median Age: 42.0
Age Group Distribution:
age_group
35-44    33.744582
45-54    29.476492
25-34    14.954985
55-64     8.169390
18-24     7.985995
65+       4.318106
<18       1.350450
Name: proportion, dtype: float64


### 2. Gender Distribution
KPIs:

- Gender Ratio: The percentage of male, female, and other gender categories.

In [12]:
# Calculate gender distribution
gender_distribution = CombainedData['gender'].value_counts(normalize=True) * 100

# Display the results
print(f"Gender Distribution:\n{gender_distribution}")


Gender Distribution:
gender
F    88.229410
M    11.587196
O     0.183394
Name: proportion, dtype: float64


### 3. Marital Status
KPIs:

- Percentage Married: The percentage of users who are married.


In [13]:
# Calculate marital status distribution
marital_status_distribution = CombainedData['married'].value_counts(normalize=True) * 100

# Display the results
print(f"Marital Status Distribution:\n{marital_status_distribution}")

Marital Status Distribution:
married
False    55.951984
True     44.048016
Name: proportion, dtype: float64


### 4. Parental Status
KPIs:

- Percentage with Children: The percentage of users who have children.

In [14]:
# Calculate parental status distribution
parental_status_distribution = CombainedData['has_children'].value_counts(normalize=True) * 100

# Display the results
print(f"Parental Status Distribution:\n{parental_status_distribution}")


Parental Status Distribution:
has_children
False    67.355785
True     32.644215
Name: proportion, dtype: float64


### 5. Geographical Distribution (Home Country and City)
KPIs:

- Top Home Countries: The most common countries of origin among users.
- Top Home Cities: The most common cities of origin among users.

In [15]:
# Calculate top home countries
top_home_countries = CombainedData['home_country'].value_counts().head(10)

# Calculate top home cities
top_home_cities = CombainedData['home_city'].value_counts().head(10)

# Display the results
print(f"Top Home Countries:\n{top_home_countries}")
print(f"Top Home Cities:\n{top_home_cities}")


Top Home Countries:
home_country
usa       4991
canada    1007
Name: count, dtype: int64
Top Home Cities:
home_city
new york       680
los angeles    325
toronto        225
chicago        195
houston        185
montreal       166
san antonio    132
phoenix        119
dallas         117
san diego      113
Name: count, dtype: int64


In [16]:
CombainedData.describe(include='all')

,user_id,birthdate,gender,married,has_children,home_country,home_city,age,age_group,latest_session,...,discount_responsiveness,total_hotel_usd_spent,total_flight_usd_spent,total_usd_spent,total_nights,scaled_hotel_ads,ads_per_km,scaled_ads_per_km,hotel_hunter_index,flight_hunter_index
count,5998.000000,5998,5998,5998,5998,5998,5998,5998.000000,5998,5998,...,5998.000000,5998.000000,5998.000000,5998.000000,5435.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000
unique,NaN,4807,3,2,2,2,105,NaN,7,152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,1986-07-11,F,False,False,usa,new york,NaN,35-44,2023-07-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5,5292,3356,4040,4991,680,NaN,2024,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,545440.695565,NaN,NaN,NaN,NaN,NaN,NaN,41.942814,NaN,NaN,...,0.047624,544.078860,1319.694518,1863.773378,10.534499,0.016829,0.000014,0.005887,0.000384,0.000235
std,64924.749760,NaN,NaN,NaN,NaN,NaN,NaN,12.047620,NaN,NaN,...,0.065957,438.584467,1838.322771,1999.683221,8.300045,0.043186,0.000046,0.019684,0.001393,0.001850
min,23557.000000,NaN,NaN,NaN,NaN,NaN,NaN,17.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,517152.000000,NaN,NaN,NaN,NaN,NaN,NaN,35.000000,NaN,NaN,...,0.000000,225.000000,398.555000,743.980000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,540542.000000,NaN,NaN,NaN,NaN,NaN,NaN,42.000000,NaN,NaN,...,0.000000,460.500000,937.080000,1466.495000,9.000000,0.000000,0.000007,0.002935,0.000000,0.000034
75%,574146.000000,NaN,NaN,NaN,NaN,NaN,NaN,49.000000,NaN,NaN,...,0.088889,775.000000,1609.827500,2375.207500,14.000000,0.013501,0.000017,0.007188,0.000060,0.000194


## Clustering our Dataset

In [21]:
df = CombainedData.copy()



# Select relevant features for clustering
features = ['total_cancellation_rate', 'average_checked_bags', 'conversion_rate',
                     'prefers_flights', 'prefers_hotels', 'prefers_both',  
                     'discount_responsiveness', 'flight_discount_proportion','average_flight_discount',
                     'hotel_discount_proportion', 'average_hotel_discount', 'both_discount_proportion',
                     'flight_hunter_index', 'hotel_hunter_index', 
                     'total_hotel_usd_spent', 'total_flight_usd_spent', 'age'
                    ]


# Convert categorical features to numeric using one-hot encoding
df_encoded = pd.get_dummies(df[features], drop_first=True)

# Scale the features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_encoded)

zscaled_columns = [feature + "_zscaled" for feature in features]
scaler = StandardScaler()
df[zscaled_columns] = scaler.fit_transform(df[features])

# Apply K-Means clustering
kmeans = KMeans(n_clusters=5,n_init=10,  random_state=42)  # Adjust the number of clusters as needed
df['cluster'] = kmeans.fit_predict(df_scaled)

# Analyze the clusters
for cluster in range(5):
    print(f"Cluster {cluster}")
    print(df[df['cluster'] == cluster][zscaled_columns].mean())
    #visual separator -----
    print('-'*50)



Cluster 0
total_cancellation_rate_zscaled      -0.215581
average_checked_bags_zscaled          0.137373
conversion_rate_zscaled               0.402056
prefers_flights_zscaled              -0.032734
prefers_hotels_zscaled               -0.212714
prefers_both_zscaled                  0.384034
discount_responsiveness_zscaled      -0.266903
flight_discount_proportion_zscaled   -0.258117
average_flight_discount_zscaled      -0.309855
hotel_discount_proportion_zscaled     0.865259
average_hotel_discount_zscaled        0.738511
both_discount_proportion_zscaled     -0.456052
flight_hunter_index_zscaled          -0.055689
hotel_hunter_index_zscaled            0.471093
total_hotel_usd_spent_zscaled         0.358436
total_flight_usd_spent_zscaled        0.061930
age_zscaled                           0.104681
dtype: float64
--------------------------------------------------
Cluster 1
total_cancellation_rate_zscaled       3.112865
average_checked_bags_zscaled          0.700739
conversion_rate_zscal